In [0]:
!pip install --upgrade deeppavlov
!pip install --upgrade transformers
!pip install -U sentence-transformers

In [0]:
from sentence_transformers import SentenceTransformer
bert=model = SentenceTransformer('distiluse-base-multilingual-cased')

sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = bert.encode(sentences)

100%|██████████| 504M/504M [00:22<00:00, 22.4MB/s]


In [0]:
 %run Pre_ini.ipynb

In [0]:
queries=dict()
with open('queries.fixed.txt') as f:
    for line in f:
        query_id,words=line.split('\t')
        queries[int(query_id)]=process_string(words).strip().replace('.',' ')

In [0]:
with open('super_query_ext.pickle','rb') as f:
    query_ext=pickle.load(f)

In [0]:
candidates,reverse_candidates=generate_candidates()

In [0]:
queries_emb=dict()
i=0
for query_id,query in queries.items():
    queries_emb[query_id]=bert.encode([query])
    if i%50==0:
        print(i)
    i+=1

0
50
100
150
200
250
300
350


In [0]:
tmp_query_ext=dict()
for key in query_ext:
    tmp_query_ext[key]=set(filter(lambda word: word not in stopwords,
                                  map(lambda x:x[1],query_ext[key])))

In [0]:
file_names=['divide_cos0.txt','divide_cos1.txt','divide_cos2.txt','divide_cos3.txt']
for n in range(0,4):
    i1=n*10000
    i2=(n+1)*10000
    with open(file_names[n],'w') as out:
        with open('forward_docs.txt','r') as f:
            beg=time()
            for i,line in enumerate(f):
                if i>=i2 or i<i1:
                    continue
                doc_id,desc,author,keywords,title,word=line.strip('\n').split('\t')

                texts=list(filter(lambda x:x and any(map(lambda x:x!=' ',x)),
                                  title.split('.')+author.split('.')+
                                  desc.split('.')+keywords.split('.')+word.split('.')
                                 ))
                
                def divide_text(text,n):
                        t=text.split()
                        return [' '.join(t[i*n:(i+1)*n]) for i in range(len(t)//n+int(len(t)%n!=0))]
                texts=[new_text for text in texts for new_text in divide_text(text,20)]
                
                texts_len=len(texts)
                old_texts=enumerate(texts)

                for query_id in reverse_candidates[int(doc_id)]:
                    query = queries[query_id]
                    s=tmp_query_ext[query_id]
                    query_emb=queries_emb[query_id][0]
                    
                    
                    texts=list(filter(lambda text:any(map(lambda word:add_cash(word) in s,
                                                        text[1].split())),old_texts))
                    
                    texts=list(map(lambda text:(text[0],' '.join(text[1].split())),texts))
                    
                    s=set()
                    def addset(s,text):
                        if text in s:
                            return False
                        else:
                            s.add(text)
                            return True
                    texts=list(filter(lambda text:addset(s,text[1]),texts))

                    out.write(doc_id+'\t'+query)
                    if texts:
                        sent_embs=bert.encode(list(map(lambda item:item[1],texts)))
                        distances = scipy.spatial.distance.cdist([query_emb], sent_embs, "cosine")[0]
                        
                        for ind in np.argsort(distances):
                            out.write('\t'+texts[ind][1]+','+str(distances[ind])+','+str(1-texts[ind][0]/texts_len))
                    out.write('\n')
                    if i%10==0:
                        print(i)


            print((time()-beg)/(i+1))